## Qiskit Gradient Framework

The gradient framework enables the evaluation of quantum gradients as well as functions thereof.
Besides standard first order gradients of expectation values of the form
$$ \langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle $$
<!--- $$ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta} $$

$$ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta^2}, $$
--->

the gradient also supports the evaluation of second order gradients (Hessians), and the Quantum Fisher Information (QFI) of pure quantum states $|\psi\left(\theta\right)\rangle$.

### First Order Gradients

Three types of first order gradients are supported by the gradient framework.
1. Gradient of an expectation value w.r.t. a coefficient of the measurement operator respectively observable $\hat{O}\left(\omega\right)$, i.e.
 $ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\omega} $
2.  Gradient of an expectation value w.r.t. a state $|\psi\left(\theta\right)\rangle$ parameter, i.e.
 $ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta} $
3.  Gradient of sampling probabilities w.r.t. a state $|\psi\left(\theta\right)\rangle$ parameter, i.e.
 $ \frac{\partial p_i}{\partial\theta} = \frac{\partial\langle\psi\left(\theta\right)|i\rangle\langle i|\psi\left(\theta\right)\rangle}{\partial\theta} $



Given a parameterized quantum state $|\psi\left(\theta\right)\rangle = V\left(\theta\right)|\psi\rangle$ with input state $|\psi\rangle$, parametrized Ansatz $V\left(\theta\right)$, and observable $\hat{O}\left(\omega\right)=\sum_{i}\omega_i\hat{O}_i$, we want to compute...

#### Gradients w.r.t. Measurement Operator Coefficients
$$ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\omega_i} = \langle\psi\left(\theta\right)|\hat{O}_i\left(\omega\right)|\psi\left(\theta\right)\rangle. $$
#### Gradients w.r.t. State Coefficients
$$ \frac{\partial p_i}{\partial\theta} = \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta}. $$

In [1]:
import numpy as np
from qiskit.aqua.operators import Z, X, I, StateFn, CircuitStateFn, SummedOp
from qiskit.aqua.operators.gradients import Gradient, NaturalGradient, QFI, Hessian
from qiskit.circuit import QuantumCircuit, QuantumRegister, Parameter, ParameterVector, ParameterExpression
from qiskit.circuit.library import EfficientSU2

In [2]:
a = Parameter('a')
b = Parameter('b')
q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rz(a, q[0])
qc.rx(b, q[0])

# Instantiate the Hamiltonian observable
coeff_0 = Parameter('c_0')
coeff_1 = Parameter('c_1')
H = (coeff_0*coeff_0*2)*X + coeff_1 * Z

# Combine the Hamiltonian observable and the state
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)

print(op)

ComposedOp([
  OperatorMeasurement(SummedOp([
    2.0*c_0**2 * X,
    1.0*c_1 * Z
  ])),
  CircuitStateFn(
        ┌───┐┌───────┐┌───────┐
  q0_0: ┤ H ├┤ RZ(a) ├┤ RX(b) ├
        └───┘└───────┘└───────┘
  )
])


In [3]:
# Define the coefficients w.r.t. we want to compute the gradient
obs_coeffs = [coeff_0, coeff_1]
circ_coeffs = [a, b]

# Define the values to be assigned to the parameters
value_dict = {coeff_0: 0.5, coeff_1: -1, a: np.pi / 4, b: np.pi}

# Convert the operator and the gradient target coefficients into the respective operator
grad = Gradient().convert(operator = op, 
                          params = obs_coeffs)
print(grad)

# Assign the parameters and evaluate the gradient
grad_result = grad.assign_parameters(value_dict).eval()
print('Gradient ', grad_result)

ListOp([
  SummedOp([
    4.0*c_0 * ComposedOp([
      OperatorMeasurement(1.00000000000000 * Z),
      CircuitStateFn(
            ┌───┐┌───────┐┌───────┐┌───┐
      q0_0: ┤ H ├┤ RZ(a) ├┤ RX(b) ├┤ H ├
            └───┘└───────┘└───────┘└───┘
      )
    ]),
    ComposedOp([
      DictMeasurement({'0': 1}),
      DictStateFn({'1': 1})
    ])
  ]),
  SummedOp([
    ComposedOp([
      DictMeasurement({'0': 1}),
      DictStateFn({'1': 1})
    ]),
    1.00000000000000 * ComposedOp([
      OperatorMeasurement(1.00000000000000 * Z),
      CircuitStateFn(
            ┌───┐┌───────┐┌───────┐
      q0_0: ┤ H ├┤ RZ(a) ├┤ RX(b) ├
            └───┘└───────┘└───────┘
      )
    ])
  ])
])
Gradient  [(1.414213562373095+1.12e-16j), 0j]


## Gradients wrt circuit parameters

##### Parameter Shift Gradients
<a id='param_shift_grad'></a>
Given a Hermitian operator $g$ with two unique eigenvalues $\pm r$ which acts as generator for a parameterized quantum gate $$G(\theta)= e^{-i\theta g}.$$
Then, quantum gradients can be computed by using eigenvalue $r$ dependent shifts to parameters. All [standard, parameterized qiskit gates](https://github.com/Qiskit/qiskit-terra/tree/master/qiskit/circuit/library/standard_gates) can be shifted with $\pi/2$, i.e.,
 $ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta} = 2 \left(\langle\psi\left(\theta+\pi/2\right)|\hat{O}\left(\omega\right)|\psi\left(\theta+\pi/2\right)\rangle - \right.$
 $\left.\partial\langle\psi\left(\theta-\pi/2\right)|\hat{O}\left(\omega\right)|\psi\left(\theta-\pi/2\right)\rangle\right).$
 Probability gradients are computed equivalently.
 
 
 ##### Finite Difference Gradients

<a id='fin_diff_grad'></a>

Unlike the other methods, finite difference gradients are numerical estimations rather than analytical values.
This implementation employs a central difference approach with $\epsilon << 1$
 $ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta} \approx \frac{1}{2\epsilon} \left(\langle\psi\left(\theta+\epsilon\right)|\hat{O}\left(\omega\right)|\psi\left(\theta+\epsilon\right)\rangle - \partial\langle\psi\left(\theta-\epsilon\right)|\hat{O}\left(\omega\right)|\psi\left(\theta-\epsilon\right)\rangle\right).$
 Probability gradients are computed equivalently.
 
 ##### Linear Combination of Unitaries Gradients
<a id='lin_comb_grad'></a>
Unitaries can be written as $U\left(\omega\right) = e^{iM\left(\omega\right)}$, where $M\left(\omega\right)$ denotes a parameterized Hermitian matrix. 
Further, Hermitian matrices can be decomposed into weighted sums of Pauli terms, i.e., $M\left(\omega\right) = \sum_pm_p\left(\omega\right)h_p$ with $m_p\left(\omega\right)\in\mathbb{R}$ and $h_p=\bigotimes\limits_{j=0}^{n-1}\sigma_{j, p}$ for $\sigma_{j, p}\in\left\{I, X, Y, Z\right\}$ acting on the $j^{\text{th}}$ qubit. Thus, the gradients of 
$U_k\left(\omega_k\right)$ are given by
\begin{equation*}
\frac{\partial U_k\left(\omega_k\right)}{\partial\omega_k} = \sum\limits_pi \frac{\partial m_{k,p}\left(\omega_k\right)}{\partial\omega_k}U_k\left(\omega_k\right)h_{k_p}.
\end{equation*}

Combining this observation with a circuit structure presented in [Simulating physical phenomena by quantum networks](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.65.042323) allows us to compute the gradient with the evaluation of a single quantum circuit.

In [4]:
# Define the Hamiltonian with fixed coefficients
H = 0.5 * X - 1 * Z
# Define the parameters w.r.t. we want to compute the gradients
params = [a, b]
# Define the values to be assigned to the parameters
value_dict = { a: np.pi / 4, b: np.pi}

# Combine the Hamiltonian observable and the state
state_grad_op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)
print(state_grad_op)


ComposedOp([
  OperatorMeasurement(SummedOp([
    0.5 * X,
    -1.0 * Z
  ])),
  CircuitStateFn(
        ┌───┐┌───────┐┌───────┐
  q0_0: ┤ H ├┤ RZ(a) ├┤ RX(b) ├
        └───┘└───────┘└───────┘
  )
])


In [5]:
state_grad = Gradient(method='param_shift').convert(operator=state_grad_op, params=a)
print(state_grad)

# Assign the parameters and evaluate the gradient
state_grad_result = state_grad.assign_parameters(value_dict).eval()
print('State gradient computed with parameter shift', state_grad_result)

SummedOp([
  0.5 * SummedOp([
    0.5 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌─────────────────────────┐┌───────┐┌───┐
      q0_0: ┤ H ├┤ RZ(a + 1.5707963267949) ├┤ RX(b) ├┤ H ├
            └───┘└─────────────────────────┘└───────┘└───┘
      )
    ]),
    -0.5 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌─────────────────────────┐┌───────┐┌───┐
      q0_0: ┤ H ├┤ RZ(a - 1.5707963267949) ├┤ RX(b) ├┤ H ├
            └───┘└─────────────────────────┘└───────┘└───┘
      )
    ])
  ]),
  -1.0 * SummedOp([
    0.5 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌─────────────────────────┐┌───────┐
      q0_0: ┤ H ├┤ RZ(a + 1.5707963267949) ├┤ RX(b) ├
            └───┘└─────────────────────────┘└───────┘
      )
    ]),
    -0.5 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌─────────────────────────┐┌───────┐
      q0_0: ┤ H ├┤ RZ(a -

In [6]:
state_grad = Gradient(method='fin_diff').convert(operator=state_grad_op, params=a)
print(state_grad)
# Assign the parameters and evaluate the gradient
state_grad_result = state_grad.assign_parameters(value_dict).eval()
print('State gradient computed with finite difference', state_grad_result)

SummedOp([
  0.5 * SummedOp([
    500000.0 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌────────────────┐┌───────┐┌───┐
      q0_0: ┤ H ├┤ RZ(a + 1.0e-6) ├┤ RX(b) ├┤ H ├
            └───┘└────────────────┘└───────┘└───┘
      )
    ]),
    -500000.0 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌────────────────┐┌───────┐┌───┐
      q0_0: ┤ H ├┤ RZ(a - 1.0e-6) ├┤ RX(b) ├┤ H ├
            └───┘└────────────────┘└───────┘└───┘
      )
    ])
  ]),
  -1.0 * SummedOp([
    500000.0 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌────────────────┐┌───────┐
      q0_0: ┤ H ├┤ RZ(a + 1.0e-6) ├┤ RX(b) ├
            └───┘└────────────────┘└───────┘
      )
    ]),
    -500000.0 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
            ┌───┐┌────────────────┐┌───────┐
      q0_0: ┤ H ├┤ RZ(a - 1.0e-6) ├┤ RX(b) ├
            └───┘└────────────────┘└───────┘
     

In [7]:
lcu_grad = Gradient(method='lin_comb').convert(operator=state_grad_op, params=a)
print(lcu_grad)
# Assign the parameters and evaluate the gradient
lcu_grad_result = lcu_grad.assign_parameters(value_dict).eval()
print('State gradient computed with LCU', lcu_grad_result)

SummedOp([
  0.5 * ComposedOp([
    OperatorMeasurement(ZZ),
    CircuitStateFn(
                  ┌───┐          ┌───────┐┌───────┐┌───┐
            q0_0: ┤ H ├────────■─┤ RZ(a) ├┤ RX(b) ├┤ H ├
                  ├───┤┌─────┐ │ └─┬───┬─┘└───────┘└───┘
    work_qubit_0: ┤ H ├┤ SDG ├─■───┤ H ├────────────────
                  └───┘└─────┘     └───┘                
    )
  ]),
  -1.0 * ComposedOp([
    OperatorMeasurement(ZZ),
    CircuitStateFn(
                  ┌───┐          ┌───────┐┌───────┐
            q0_0: ┤ H ├────────■─┤ RZ(a) ├┤ RX(b) ├
                  ├───┤┌─────┐ │ └─┬───┬─┘└───────┘
    work_qubit_0: ┤ H ├┤ SDG ├─■───┤ H ├───────────
                  └───┘└─────┘     └───┘           
    )
  ])
])
State gradient computed with LCU (-0.35355339059327373-2.385e-16j)


In [8]:
print('State gradient computed with parameter shift    ', np.real(state_grad_result))
print('State gradient computed with finite difference  ', np.real(state_grad_result))
print('State gradient computed with LCU                ', np.real(lcu_grad_result))

State gradient computed with parameter shift     -0.3535533905669581
State gradient computed with finite difference   -0.3535533905669581
State gradient computed with LCU                 -0.35355339059327373


## Natural Gradient

A special type of first order gradient is the natural gradient which has proven itself useful in classical machine learning and is already being studied in the quantum context. This quantity represents a gradient that is 'rescaled' with the inverse Quantum Fisher Information matrix
$$ QFI ^{-1} \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta}.$$

Instead of inverting the QFI, one can also use a least-square solver with or without regularization to solve

$$ QFI x = \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta}.$$

The implementation supports ridge and lasso regularization with automatic search for a good parameter using [L-curve corner search](https://arxiv.org/pdf/1608.04571.pdf) as well as two types of perturbations of the diagonal elements of the QFI.

The natural gradient can be used instead of the standard gradient with any gradient-based optimizer and/or ODE solver.

In [9]:
# A regularization method can be chosen, e.g. ridge or lasso with automatic 
# parameter search
nat_grad = NaturalGradient(grad_method='lin_comb', regularization='ridge').convert(
    operator=state_grad_op, params=params)

# Assign the parameters and evaluate the gradient
nat_grad_result = nat_grad.assign_parameters(value_dict).eval()
print('Natural gradient computed with linear combination of unitaries', nat_grad_result)

Natural gradient computed with linear combination of unitaries [-2.17418416  1.90241114]


### Second Order Gradients

Four types of second order gradients are supported by the gradient framework.
1. Gradient of an expectation value w.r.t. a coefficient of the measurement operator respectively observable $\hat{O}\left(\omega\right)$, i.e.
 $ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\omega^2} $
2.  Gradient of an expectation value w.r.t. a state $|\psi\left(\theta\right)\rangle$ parameter, i.e.
 $ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta^2} $
3.  Gradient of sampling probabilities w.r.t. a state $|\psi\left(\theta\right)\rangle$ parameter, i.e.
 $ \frac{\partial^2 p_i}{\partial\theta^2} = \frac{\partial^2\langle\psi\left(\theta\right)|i\rangle\langle i|\psi\left(\theta\right)\rangle}{\partial\theta^2} $
4.  Gradient of an expectation value w.r.t. a state $|\psi\left(\theta\right)\rangle$ parameter and a coefficient of the measurement operator respectively observable $\hat{O}\left(\omega\right)$, i.e.
 $ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta\partial\omega} $

In [10]:
# Instantiate the Hamiltonian observable
coeff_0 = Parameter('c_0')
coeff_1 = Parameter('c_1')
H = coeff_0*coeff_1*coeff_1*X

# Instantiate the quantum state with two parameters
a = Parameter('a')
b = Parameter('b')

q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rz(a, q[0])
qc.rx(b, q[0])

# Combine the Hamiltonian observable and the state
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)

# Define the coefficient tuple w.r.t. which we want to compute the gradient
hessian_coeffs = (coeff_0, coeff_1)

# Convert the operator and the hessian target coefficients into the respective operator
hessian = Hessian().convert(operator = op, 
                            params = [coeff_0, coeff_1])

# Define the values to be assigned to the parameters
value_dict = {coeff_0: 0.5, coeff_1: -1, a: np.pi / 4, b: np.pi/4}

# Assign the parameters and evaluate the gradient
hessian_result = hessian.assign_parameters(value_dict).eval()
print('Hessian \n', np.real(np.array(hessian_result)))

Hessian 
 [[ 0.         -1.41421356]
 [-1.41421356  0.70710678]]


In [11]:
params = [a,b]

# Get the operator object representing the Hessian
state_hess = Hessian(method='param_shift').convert(operator=op, params=params)
# Assign the parameters and evaluate the Hessian
hessian_result = state_hess.assign_parameters(value_dict).eval()

# Get the operator object representing the Hessian using finite difference
state_hess = Hessian(method='fin_diff').convert(operator=op, params=params)
# Assign the parameters and evaluate the Hessian
hessian_result = state_hess.assign_parameters(value_dict).eval()

# Get the operator object representing the Hessian
prob_hess = Hessian(method='lin_comb').convert(operator=op, params=params)
# Assign the parameters and evaluate the Hessian
hessian_result = prob_hess.assign_parameters(value_dict).eval()


print('Hessian computed using the parameter shift method\n', np.real(np.array(hessian_result)))
print('Hessian computed with finite difference\n', np.real(np.array(hessian_result)))
print('Hessian computed using the linear combination of unitaries method\n', np.real(np.array(hessian_result)))

Hessian computed using the parameter shift method
 [[-0.35355339  0.        ]
 [ 0.          0.        ]]
Hessian computed with finite difference
 [[-0.35355339  0.        ]
 [ 0.          0.        ]]
Hessian computed using the linear combination of unitaries method
 [[-0.35355339  0.        ]
 [ 0.          0.        ]]


# QFI
The Quantum Fisher Information is a metric tensor which is representative for the representation capacity of a 
parameterized quantum state $|\psi\left(\theta\right)\rangle = V\left(\theta\right)|\psi\rangle$ with input state $|\psi\rangle$, parametrized Ansatz $V\left(\theta\right)$.

The entries of the QFI for a pure state reads

$$
QFI_{kl} = 4 * \text{Re}\left[\langle\partial_k\psi|\partial_l|psi\rangle-\langle\partial_k\psi|\psi\rangle\langle\psi|\partial_l\psi\rangle \right].$$

#### Full QFI
To compute the full QFI, we use a working qubit as well as intercepting controlled gates. See e.g. [Variational ansatz-based quantum simulation of imaginary time evolution ](https://www.nature.com/articles/s41534-019-0187-2).

In [12]:
# Wrap the quantum circuit into a CircuitStateFn
state = CircuitStateFn(primitive=qc, coeff=1.)

# Convert the state and the parameters into the operator object that represents the QFI 
qfi = QFI().convert(operator=state, params=params)
# Define the values for which the QFI is to be computed
values_dict = {a: np.pi / 4, b: 0.1}

# Assign the parameters and evaluate the QFI
qfi_result = qfi.assign_parameters(values_dict).eval()
print('full  QFI \n', np.real(np.array(qfi_result)))

full  QFI 
 [[ 1.00000000e+00 -1.97989899e-17]
 [-1.97989899e-17  5.00000000e-01]]


#### Block-diagonal and Diagonal Approximation
A block-diagonal resp. diagonal approximation of the QFI can be computed without additional working qubits.
This implementation requires the unrolling into Pauli rotations and unparameterized Gates.

In [13]:
# Convert the state and the parameters into the operator object that represents the QFI 
# and set the approximation to 'block_diagonal'
qfi = QFI('overlap').convert(operator=state, params=params)

# Assign the parameters and evaluate the QFI
qfi_result = qfi.assign_parameters(values_dict).eval()
print('Dlock-diagonal QFI \n', np.real(np.array(qfi_result)))

# Convert the state and the parameters into the operator object that represents the QFI 
# and set the approximation to 'diagonal'
qfi = QFI('overlap', 'diag').convert(operator=state, params=params)

# Assign the parameters and evaluate the QFI
qfi_result = qfi.assign_parameters(values_dict).eval()
print('Diagonal QFI \n', np.real(np.array(qfi_result)))

Dlock-diagonal QFI 
 [[1.  0. ]
 [0.  0.5]]
Diagonal QFI 
 [[1.  0. ]
 [0.  0.5]]


### Application Examples

#### VQE with first and second order gradient based optimization

Note: The operator flow's eval method is used to evaluate the function, gradient and Hessian values but we could also use any Backend/QuantumInstance.

In [14]:
from qiskit.aqua.operators import I, X, Z
from qiskit.circuit import QuantumCircuit, ParameterVector
from scipy.optimize import minimize

h2_hamiltonian = -1.05 * (I ^ I) + 0.39 * (I ^ Z) - 0.39 * (Z ^ I) - 0.01 * (Z ^ Z) + 0.18 * (X ^ X)
h2_energy = -1.85727503

# Define the Ansatz
wavefunction = QuantumCircuit(2)
params = ParameterVector('theta', length=8)
it = iter(params)
wavefunction.ry(next(it), 0)
wavefunction.ry(next(it), 1)
wavefunction.rz(next(it), 0)
wavefunction.rz(next(it), 1)
wavefunction.cx(0, 1)
wavefunction.ry(next(it), 0)
wavefunction.ry(next(it), 1)
wavefunction.rz(next(it), 0)
wavefunction.rz(next(it), 1)

op = ~StateFn(h2_hamiltonian) @ StateFn(wavefunction)

In [15]:
def fun(param_values):
    param_dict = dict(zip(params, param_values)) 
    return op.assign_parameters(param_dict).eval()


jac = Gradient(method = 'param_shift').gradient_wrapper(op, bind_params = params)
# param_tuples = [[(param0, param1) for param0 in params] for param1 in params]
hess = Hessian(method = 'param_shift').gradient_wrapper(op, bind_params = params)

result = minimize(fun, np.random.rand(len(params)), method='Newton-CG', jac=jac, hess=hess, options={'maxiter': 10, 'xtol': 1e-8})
print('VQE using the gradient and the Hessian:', result['fun'], 'Reference:', h2_energy)

result = minimize(fun, np.random.rand(len(params)), method='Newton-CG', jac=jac, options={'maxiter': 10, 'xtol': 1e-8})
print('VQE using the gradient:', result['fun'], 'Reference:', h2_energy)

# result = minimize(fun, np.random.rand(len(params)), method='CG', options={'maxiter': 10, 'tol': 1e-8})

# print('gradient-free VQE:', result['fun'], 'Reference:', h2_energy)

/Users/brycefuller/anaconda3/envs/QiskitDevEnv/lib/python3.7/site-packages/scipy/optimize/linesearch.py:172: ComplexWarning: Casting complex values to real discards the imaginary part
  amin, amax, isave, dsave)


VQE using the gradient and the Hessian: -1.8404998438468294 Reference: -1.85727503
VQE using the gradient: -1.8404998438462055 Reference: -1.85727503


In [16]:
from qiskit.aqua.operators import CircuitSampler
from qiskit import Aer
def fun(param_values):
    param_dict = dict(zip(params, param_values))
    param_dict = {k: [v] for k, v in param_dict.items()}
    sampler = CircuitSampler(backend=Aer.get_backend('qasm_simulator')).convert(op, params=param_dict)
    return sampler.eval()


jac = Gradient(method = 'param_shift').gradient_wrapper(op, bind_params = params, backend=Aer.get_backend('qasm_simulator'))
hess = Hessian(method = 'param_shift').gradient_wrapper(op, bind_params = params, backend=Aer.get_backend('qasm_simulator'))

result = minimize(fun, np.random.rand(len(params)), method='Newton-CG', jac=jac, hess=hess, 
                  options={'maxiter': 10, 'xtol': 1e-8})

print('VQE using the gradient and the Hessian:', result['fun'])

result = minimize(fun, np.random.rand(len(params)), method='Newton-CG', jac=jac, options={'maxiter': 10, 'xtol': 1e-8})

print('VQE using the gradient:', result['fun'])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

#### Gibbs state preparation using Variational Quantum Imaginary Time Evolution (VarQITE)

In [ ]:
from qiskit.circuit.library import RealAmplitudes

# Temperature
T = 5

# Evolution time
t =  1/(2*T)

# Define the model Hamiltonian
H = SummedOp([0.3 * Z^Z^ I^I, 0.2 * Z^I^ I^I, - 0.5 * I ^ Z^ I^I])

# Instantiate the model ansatz
depth = 1
entangler_map = [[i+1, i] for i in range(H.num_qubits - 1)]
ansatz = EfficientSU2(4, reps=depth, entanglement = entangler_map)
qr = ansatz.qregs[0]
for i in range(int(len(qr)/2)):
    ansatz.cx(qr[i], qr[i+int(len(qr)/2)])
    
# Initialize the Ansatz parameters
param_values_init = np.zeros(2* H.num_qubits * (depth + 1))
for j in range(2 * H.num_qubits * depth, int(len(param_values_init) - H.num_qubits - 2)):
    param_values_init[int(j)] = np.pi/2.

In [ ]:
# Define the Hamiltonian as observable w.r.t. the wavefunction generated by the Ansatz    
op = ~StateFn(H) @ CircuitStateFn(ansatz)

# Define the discretization grid of the time steps
num_time_steps = 10
time_steps = np.linspace(0, t, num_time_steps)

# Convert the operator that holds the Hamiltonian and ansatz into a NaturalGradient operator 
nat_grad = NaturalGradient(grad_method = 'lin_comb', regularization = 'ridge').convert(op, ansatz.ordered_parameters)

param_values = param_values_init
# Propagate the Ansatz parameters step by step according to the explicit Euler method
for step in time_steps:
    param_dict = dict(zip(ansatz.ordered_parameters, param_values))
    nat_grad_result = np.real(nat_grad.assign_parameters(param_dict).eval())
    param_values = list(np.subtract(param_values, t/num_time_steps * np.real(nat_grad_result)))

param_dict_final = dict(zip(ansatz.ordered_parameters, param_values))

In [ ]:
print(CircuitStateFn(ansatz).assign_parameters(param_dict_final).eval().primitive.data)  

In [ ]:
param_values